In [1]:
import sys
import os
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [3]:
# Loading Config
from utils.config import load_yaml_config, merge_configs, dict_to_namespace
import numpy as np

default_yaml = "../utils/configs/default.yaml"
problem_yaml = "../utils/configs/problem_1.yaml"
category_yaml = "../utils/configs/experiment_trn.yaml"

custom_config_files = [default_yaml, problem_yaml, category_yaml] # Add more config files if needed
configs = [load_yaml_config(config_file) for config_file in custom_config_files]
yaml = merge_configs(*configs)

## Here you can modify the yaml object to change the configuration
## Example: yaml.seed = 123, yaml.max_num_tasks = 10
config = dict_to_namespace(yaml)

In [4]:
import torch
import random

# Set random seeds for reproducibility

random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)

In [5]:
from env.cades_env import CadesEnv
from stable_baselines3.common.env_checker import check_env

# Initialize and check the environment
env = CadesEnv(config)
check_env(env)

/home/ediss5/anaconda3/envs/cades/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:190: UserWarning: Your observation communications has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [6]:
config.train = False
config.evaluate = True

In [10]:
from models.ppo import PPOModel

model = PPOModel.load("../../experiments/models/p1/trn/early_term_1000", env, config)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
%ls

chain_comm.ipynb  evaluate.ipynb


In [8]:
# redefine model's predict method to use the heuristic
from heuristics.nf import NextFitHeuristic

heuristic = NextFitHeuristic(env)
model.model.predict = heuristic.predict

In [11]:
model.evaluate_multiple()

{'mean_episode_reward': 43.38206944444444,
 'mean_episode_length': 11.78,
 'mean_inference_time': 0.06502962350845337,
 'termination_cause': {'success': 96.0,
  'duplicate_pick': 2.0,
  'node_overflow': 0.0,
  'duplicate_critical_pick': 2.0},
 'mean_metrics': {'avg_node_occupancy': 66.66999999999999,
  'avg_active_node_occupancy': 66.66999999999999,
  'message_channel_occupancy': 0.0,
  'empty_nodes': 0.0}}